# for testing conda virtual environment creation

## 0. Installing and Importing Dependencies

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

2023-06-26 21:40:58.126373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1. Loading model